In [ ]:
from Problem import Problem
import networkx as nx
from src.genetic_algorithm import GeneticAlgorithm
from itertools import product
import numpy as np
import os
import random

result_dir = "experiments"
os.makedirs(result_dir, exist_ok=True)
num_experiments = len(os.listdir(result_dir))

In [ ]:
# Combinations 1
number_cities = [100, 1000]
alphas = [1, 2]
betas = [1, 2]
densities = [0.2, 1]

# Combinations 2
# number_cities = [100, 1000]
# alphas = [0.001, 0.002, 0.006]
# betas = [1, 2]
# densities = [0.2, 1]

In [ ]:
def check_solution(sol: list[tuple[int, float]], p, best_solution):
    # check a direct edge exists between consecutive cities
    for i in range(len(sol)-1):
        if not p.graph.has_edge(sol[i][0], sol[i+1][0]):
            return False
    #check gold is taken from all cities
    gold_collected = set(city for city, gold in sol if gold > 0)
    all_cities = set(range(1, p.graph.number_of_nodes()))  
    values = set(best_solution.solution)
    if gold_collected != all_cities:
        return False
    # check solution values are correct
    for v in values:
        if v != best_solution.solution[v-1]:
            return False
    return True

In [ ]:
for i, (num_cities, alpha, beta, density) in enumerate(product(number_cities, alphas, betas, densities)):
    random.seed(42)
    np.random.seed(42)
    problem = Problem(num_cities, alpha=alpha, beta=beta, density=density)
    print(f"Testing with {num_cities} cities, alpha={alpha}, beta={beta}, density={density}")
    G = problem.graph
    pos_dict = {k: v['pos'] for k, v in G.nodes(data=True)}
    gold_dict = {n: data['gold'] for n, data in G.nodes(data=True)}
    paths_dict = nx.shortest_path(problem.graph, source=0, weight='dist')
    population_size = 100 if num_cities == 100 else 50
    max_generations = 50 if num_cities == 100 else 25
    density = nx.density(G)
    pd_param = 0.8 if density < 0.5 else 0.5
    GA = GeneticAlgorithm(problem, population_size=population_size, max_generations=max_generations, mutation_rate=0.5, mutation_choice=0.5, pd_param=pd_param)
    best_solution, best_cost = GA.solve()
    base_cost = problem.baseline()
    valid = check_solution(best_solution.format_solution(), problem, best_solution)
    print(f"Solution valid: {valid}")
    print(f"Baseline solution: {base_cost}")
    print(f"GA solution: {best_cost}")
    log_dir = f"{result_dir}/exp_{i+num_experiments}"
    os.makedirs(log_dir, exist_ok=True)
    GA.log(log_dir=log_dir, baseline_cost=base_cost)
    GA.plot_history()